This is a noteboook used to generate the speaker embeddings with the  GE2E model.

In [26]:
import os
import numpy as np
import torch
from glob import glob
from tqdm import tqdm

In [3]:
# Clone encoder 
#!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git
! git clone https://github.com/Edresson/GE2E-Speaker-Encoder.git
os.chdir('GE2E-Speaker-Encoder/')

fatal: destination path 'GE2E-Speaker-Encoder' already exists and is not an empty directory.


In [6]:
#Install voxceleb_trainer Requeriments
!python -m pip install umap-learn visdom webrtcvad librosa>=0.5.1 matplotlib>=2.0.2 numpy>=1.14.0  scipy>=1.0.0  tqdm sounddevice Unidecode inflect multiprocess numba

In [9]:
#Download encoder Checkpoint
!wget https://github.com/Edresson/Real-Time-Voice-Cloning/releases/download/checkpoints/pretrained.zip
!unzip pretrained.zip

Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/pretrained.pt  


In [10]:
from encoder import inference as encoder
from encoder.params_model import model_embedding_size as speaker_embedding_size
from pathlib import Path

In [13]:
print("Preparing the encoder, the synthesizer and the vocoder...")
encoder.load_model(Path('encoder/saved_models/pretrained.pt'))
print("Testing your configuration with small inputs.")
# Forward an audio waveform of zeroes that lasts 1 second. Notice how we can get the encoder's
# sampling rate, which may differ.
# If you're unfamiliar with digital audio, know that it is encoded as an array of floats 
# (or sometimes integers, but mostly floats in this projects) ranging from -1 to 1.
# The sampling rate is the number of values (samples) recorded per second, it is set to
# 16000 for the encoder. Creating an array of length <sampling_rate> will always correspond 
# to an audio of 1 second.
print("\tTesting the encoder...")

wav = np.zeros(encoder.sampling_rate)    
embed = encoder.embed_utterance(wav)
print(embed.shape)

# Embeddings are L2-normalized (this isn't important here, but if you want to make your own 
# embeddings it will be).
#embed /= np.linalg.norm(embed) # for random embedding


Preparing the encoder, the synthesizer and the vocoder...
Loaded encoder "pretrained.pt" trained to step 1564501
Testing your configuration with small inputs.
	Testing the encoder...
(256,)


In [34]:
! ls ../

GE2E-CorentinJ-ExtractSpeakerEmbeddings-by-sample.ipynb
GE2E-Speaker-Encoder
Speech2Phone-ExtractSpeakerEmbeddings-by-sample.ipynb


In [43]:
# Set constants
DATA_ROOT_PATH = '../../../../LibriSpeech/voicefilter_data/'
TRAIN_DATA = os.path.join(DATA_ROOT_PATH, 'train')
TEST_DATA = os.path.join(DATA_ROOT_PATH, 'test')
glob_re_wav_emb = '*-ref_emb.wav'
glob_re_emb = '*-emb.pt'

In [36]:
os.listdir(TRAIN_DATA)

['000152-ref_emb.wav',
 '000152-mixed.wav',
 '000131-target.wav',
 '000107-ref_emb.wav',
 '000063-mixed.wav',
 '000070-mixed.wav',
 '000159-ref_emb.wav',
 '000048-target.wav',
 '000048-ref_emb.wav',
 '000030-mixed.wav',
 '000149-mixed.wav',
 '000065-mixed.wav',
 '000094-mixed.wav',
 '000156-mixed.wav',
 '000107-mixed.wav',
 '000110-target.wav',
 '000087-target.wav',
 '000157-target.wav',
 '000055-ref_emb.wav',
 '000126-target.wav',
 '000065-ref_emb.wav',
 '000158-target.wav',
 '000025-mixed.wav',
 '000121-ref_emb.wav',
 '000083-ref_emb.wav',
 '000094-target.wav',
 '000015-target.wav',
 '000055-target.wav',
 '000123-ref_emb.wav',
 '000148-mixed.wav',
 '000024-target.wav',
 '000025-ref_emb.wav',
 '000063-ref_emb.wav',
 '000048-mixed.wav',
 '000126-mixed.wav',
 '000018-mixed.wav',
 '000123-mixed.wav',
 '000087-mixed.wav',
 '000110-ref_emb.wav',
 '000157-ref_emb.wav',
 '000149-ref_emb.wav',
 '000157-mixed.wav',
 '000077-ref_emb.wav',
 '000077-mixed.wav',
 '000148-target.wav',
 '000030-targ

In [37]:
os.listdir(TEST_DATA)

[]

In [44]:
#Preprocess dataset
train_files = sorted(glob(os.path.join(TRAIN_DATA, glob_re_wav_emb)))
test_files = sorted(glob(os.path.join(TEST_DATA, glob_re_wav_emb)))

if len(train_files) == 0 or len(test_files):
    print("check train and test path files not in directory")
files  = train_files+test_files
      

for i in tqdm(range(len(files))):
    wave_file_path = files[i]
    wav_file_name = os.path.basename(wave_file_path)
    # Extract Embedding
    preprocessed_wav = encoder.preprocess_wav(wave_file_path)
    file_embedding = encoder.embed_utterance(preprocessed_wav)
    output_name = wave_file_path.replace(glob_re_wav_emb.replace('*',''),'')+glob_re_emb.replace('*','')
    torch.save(torch.from_numpy(file_embedding.reshape(-1)), output_name)

100%|██████████| 36/36 [00:10<00:00,  3.46it/s]


In [40]:
!rm *.wav

In [41]:
!ls

=0.5.1	     encoder		    requirements.txt
=1.0.0	     encoder_preprocess.py  synthesizer
=1.14.0      encoder_train.py	    synthesizer_preprocess_audio.py
=2.0.2	     pretrained.zip	    vocoder
demo_cli.py  README.md
